## A (A-star) algorithm*

In [ ]:
from typing import List, Tuple, Dict, Callable
from math import sqrt
import math 
from heapq import heappop, heappush

class Graph:
    '''Class to represent a Graph, as a list of weighted nodes and edges.'''

    def __init__(self):
        '''
        Function to initialize a Graph object
        TODO: You can modify it to suite your needs
        '''
        self.nodes: Dict[Tuple, int] = {}
        self.edges: Dict[Tuple, Dict[Tuple, int]] = {}
        self.max: List[Tuple[Tuple[int, int], int]] = \
            [((-1, -1), -1), ((-1, -1), -1)]
        self.weight_list: List[int] = []


    def add_node(self, node_id: Tuple[int, int], weight: int):
        '''
        You are free to choose your own way to represent nodes and edges.
        Im gonna use this form self.nodes: Dict[Tuple, int] = {}
        '''
        self.nodes[node_id] = weight 
        
        self.weight_list.append(weight)
        self.weight_list.sort(reverse=True)

        for k, v  in self.nodes.items():
            if v == self.weight_list[0]:
                self.max[0] = (k, v)
                break

        if len(self.weight_list) > 1:
            for k, v  in self.nodes.items():
                if v == self.weight_list[1]:
                    self.max[1] = (k, v)
                    break
                    


    def add_edge(self,
                 source_id: Tuple[int, int],
                 end_id: Tuple[int, int],
                 cost: int):
        '''Function to add an edge to a Graph object.
        ### Parameters
        `source_id` : tuple representing the ID of the source node of the edge.
        `end_id` : tuple representing the ID of the end node of the edge.
        `cost` : int representing the cost of an edge in the graph.
        You are free to choose your own way to represent nodes and edges.
        !! Notice: Take appropriate measures to create an undirected graph
        '''
        node_A = source_id
        node_B = end_id

        if node_A not in self.edges:
            self.edges[node_A] = {}
        self.edges[node_A][node_B] = cost

        #for B to A 
        if node_B not in self.edges:
            self.edges[node_B] = {}
        self.edges[node_B][node_A] = cost
        

        

    def astar_shortest_path(self,
                            source_id: Tuple[int, int],
                            end_id: Tuple[int, int],
                            heuristic: Callable):
        '''Function to return the shortest path between two nodes in a Graph.
        ### Parameters
        `G` : object representing a Graph.
        `source_id` : variable representing the ID of the source node.
        `end_id` : variable representing the ID of the end node.
        `heuristic` : heuristic function to compute the distance
            between two nodes.
        ### Return
        A list of the nodes of the shortest path as grid coordinate tuples
            e.g. `[(x1, y1), (x2, y2), ..]`
        '''

        open_list = [(0, source_id)]
        g_values = {source_id: 0}
        parent = {source_id: None} #The source node's parent is set to None in the parents dictionary, so when current_node becomes None, the loop stops.
        
        while open_list:
            #print(open_list)
            current_f, current_node = heappop(open_list)
            #print('current node', current_node)
            if current_node == end_id:
                path = [] 
                while current_node:
                    path.append(current_node)
                    current_node = parent[current_node]
                    
                #print(path,'before reverse')
                return path[::-1]
            #                    {(1, 1): {(0, 1): 1, (1, 0): 1}} --> (0, 1): 1, (1, 0): 1
            for neighbor, cost in self.edges.get(current_node, {}).items():
                #print(neighbor, cost, "-->neighbor, cost ")
                total_g = g_values[current_node] + cost
                if total_g < g_values.get(neighbor, float('inf')):
                    g_values[neighbor] = total_g
                    f_value = total_g + heuristic(neighbor, end_id) #heuristic
                    #print(open_list,'before push')
                    heappush(open_list, (f_value, neighbor))
                   # print(open_list,'after push')
                    parent[neighbor] = current_node
        return []
        

def build_Graph(nodes_l: List[Tuple[Tuple[int, int], int]],
                edges_l: List[Tuple[Tuple[int, int], Tuple[int, int], int]]):
    '''Function to build a grid-like Graph object from the input data.
    ### Parameters
    nodes_l : list of nodes.
        i.e. [((x1, y1), weight), ((x2, y2), weight), ...]
    edges : list of edges, each represented as source and end node coordinates,
        and edge cost.
        i.e. [((x1, y1), (x2, y2), cost),((x3, y3), (x4, y4), cost), ...]
    !! Notice: Take appropriate measures to create an undirected graph
    ### Return
    A Graph object.
    '''
    G = Graph()

    for node, w in nodes_l:
        G.add_node(node, weight=w)

    for node1, node2, c in edges_l:
        G.add_edge(node1, node2, cost=c)

    return G


def heuristic(u: Tuple[int, int], v: Tuple[int, int]):
    '''Function to compute the Euclidean distance between two nodes.
    ### Parameters
    `u` : first node (x1, y1).
    `v` : second node (x2, y2).
    `heuristic` : heuristic function to compute the distance between two nodes.
    ### Return
    The distance.
    '''
    e_dist = sqrt((v[0]-u[0])**2 + (v[1]-u[1])**2)
    return e_dist
    # TODO: Compute and return the Euclidean distance between u and v


def shortest_path(nodes_l: List[Tuple[Tuple[int, int], int]],
                  edges_l: List[Tuple[Tuple[int, int], Tuple[int, int], int]]
                  ) -> List[Tuple[int, int]]:
    '''
    Given a grid-like weighted graph,
        find the shortest path between the two nodes with max weight.
    Because of the grid structure of the graphs, you must implement A*.
    The heuristic to use is the Euclidean distance between
        the source and the end node of the shortest path.
    We always assume there is a path between the two nodes,
        all nodes have different weight,
        all edges have the same cost.
    ### Parameters
    `nodes_l`: List of tuples of the graph nodes
        e.g. `[((x1, y1), weight1)), ((x2, y2), weight1)), ...]`
    `edges_l`: List of the edges of the graph and their cost as `tuples`
        e.g. `[((x1, y2), (x2, y2), cost), ((x3, y3), (x4, y4), cost), ..]
    ### Return
    A list of nodes as tuples of the shortest path as grid coordinate tuples
        e.g. `[(x1, y1), (x2, y2), ..])`
    '''

    # TODO: Build a grid graph from the input nodes and edges
    G: Graph = build_Graph(nodes_l, edges_l)
    #print('G max',G.max[0][0], "--> because they are the max weighted nodes")
    start_node_max_weight = G.max[0][0]
    #print('source_id from shortest_path function', start_node_max_weight)
    end_node_second_max_weight = G.max[1][0]
   # print(G.edges, "--> EDGES")
    s_path = G.astar_shortest_path(start_node_max_weight, end_node_second_max_weight, heuristic)
    return s_path





## Dijkstra's Algorithm 

In [ ]:
import heapq

from typing import List, Tuple, Dict


class Graph:
    '''Class to represent a Graph, as a list of weighted nodes and edges.'''

    def __init__(self):
        '''Function to initialize a Graph object
        TODO: You can modify it to suite your needs
        '''
        self.nodes: List[int] = []
        self.edges: Dict[int, Dict[int, int]] = {}

    def add_node(self, node_id: int):
        '''Function to add a node to a Graph object.
        ### Parameters
        `node_id` : `int` representing the ID of a node in the graph.
        You are free to choose your own way to represent nodes and edges.
        '''
        self.nodes = self.nodes + [node_id] 
        #print(self.nodes)

    def add_edge(self,
                 source_id: int,
                 end_id: int,
                 cost: int):
        '''Function to add an edge to a Graph object.
        ### Parameters
        `source_id` : `int` representing the ID of the source node of the edge.
        `end_id` : `int` representing the ID of the end node of the edge.
        `cost` : `int` representing the cost of an edge in the graph.
        You are free to choose your own way to represent nodes and edges.
        !! Notice: Take appropriate measures to create an undirected graph
        '''
        if source_id not in self.edges:
            self.edges[source_id] = {}
        self.edges[source_id][end_id] = cost

        # Since it's an undirected graph
        if end_id not in self.edges:
            self.edges[end_id] = {}
        self.edges[end_id][source_id] = cost

 
def build_Graph(nodes_l: List[int],
                edges_l: List[Tuple[int, int, int]]):
    '''Function to build a grid-like Graph object from the input data.
    ### Parameters
    `nodes_l` : list of nodes ids.
    `edges_l` : list of edges, each represented as source and end nodes,
        and edge cost.
        i.e. [(u, v, cost),(u1, v1, cost), ...]
    !! Notice: Take appropriate measures to create an undirected graph
    ### Return
    A Graph object.
    '''
    G = Graph()

    for node in nodes_l:
        G.add_node(node)

    for node1, node2, c in edges_l:
        G.add_edge(node1, node2, cost=c)

    return G


def dijkstra(start_node, nodes_l, edges_l):

    G = build_Graph(nodes_l, edges_l)

    shortest_distance = {}
    for node in G.nodes:
        shortest_distance[node] = float('inf') 

    shortest_distance[start_node] = 0

    visited = {}
    for node in G.nodes:
        visited[node] = 'not visited'

  
    priority_queue = [(0, start_node)]  # (cost, node)

    while priority_queue:
        current_dist, current_node = heapq.heappop(priority_queue)
        
        if visited[current_node] == 'visited':
            continue

        visited[current_node] = 'visited'

        # Relax the edges
        for neighbor, weight in G.edges[current_node].items():
            if visited[neighbor] == 'not visited':
                new_distance = shortest_distance[current_node] + weight

                if new_distance < shortest_distance[neighbor]:
                    shortest_distance[neighbor] = new_distance
                    heapq.heappush(priority_queue, (new_distance, neighbor))


    return visited



def most_central(nodes_l, edges_l) -> int:
    G = build_Graph(nodes_l, edges_l)
    centrality = {node: 0 for node in G.nodes}

    for start_node in G.nodes:
        nodes_visited = dijkstra(start_node, nodes_l, edges_l)
        for visited_node in nodes_visited.keys():
            centrality[visited_node] += 1

        
    # Subtract one for the paths originating from the node itself
    for node in centrality:
        centrality[node] -= 1

    central_node = max(centrality, key=centrality.get)
    return central_node




if __name__ == "__main__":
    # Define nodes
    nodes_l = [1, 2, 3, 4, 5]

    # Define edges
    edges_l = [
        (1, 2, 3),
        (2, 3, 1),
        (2, 4, 2),
        (3, 5, 1)
    ]

    # Call the function and print the result
    central_node = most_central(nodes_l, edges_l)

## Prim's Algorithm

In [ ]:

from typing import List, Tuple
import heapq


def min_spanning_tree(nodes_l: List[int],
                      edges_l: List[Tuple[int, int, int]]
                      ) -> List[Tuple[int, int, int]]:
    '''
    Find the minimum spanning tree
    ### Parameters
    `nodes_l`: List of `integer` graph nodes
    `edges_l`: List of the edges of the graph and their cost as `tuples`
        e.g. `[(u1, v1, cost1), (u2, v2, cost2), ...]`
    ### Return
    The edges of the minimum spanning tree and their weight,
        same format as `edges_l`.
        The order of the return list does not matter:
            e.g. `[(u1,v1,c1), (u2,v2,c2)] == [(u2,v2,c2), (u1,v1,c1)]`
        As an undirected graph, the *direction* of the edges does not matter:
            e.g. `[(u, v, c)] == [(v, u, c)]`
    '''

    mst_edges = [] #collects the edges of the minimum spanning tree and their weight

    visited = set() #keep track on the visited nodes
    heapq.heapify(nodes_l) #heapify it to have min to max order 
    start_node = nodes_l[0] #first node in the list will ebe the startign node 

    visited.add(start_node)
    

    priority_queue  = [] # store edges, prioritized by their weight

   
    for src_node, dest_node, weight in edges_l:  #loop through the edge list find the source node 
        if src_node == start_node:
            priority_queue.append((weight, src_node, dest_node)) #adding the first node path(s)
    

    # Converting the list into a min-heap
    heapq.heapify(priority_queue)

    
    while priority_queue :
        weight, src_node, dest_node = heapq.heappop( priority_queue ) #popping the min node, edge and its weight 
     
       
        if dest_node not in visited:
            visited.add(dest_node) 
            mst_edges.append((src_node, dest_node, weight)) #adding destination node its source_node and weight in MST
            
            # Adding edges of newly visited node
            for src_edge, dest_edge, edge_weight in edges_l:

                if src_edge == dest_node and dest_edge not in visited:
                    heapq.heappush( priority_queue , (edge_weight, dest_node, dest_edge))

                elif dest_edge == dest_node and src_edge not in visited:
                    heapq.heappush( priority_queue , (edge_weight, dest_node, src_edge))
    
    return mst_edges